In [1]:
from pathlib import Path
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy import sparse
from scipy.stats import norm

# Чтение данных

Я не стал добавлять код для обработки данных в этот ноутбук, он представлен в `data_processing.py`. В нем выполняются практически такие же операции, как и в первых 2 заданиях, в результате генерируются и сохранаются на диск 2 словаря. Их ключи -- названия признаков или групп признаков в случае one-hot кодирования, значения -- 2-мерные массивы значений. Прочитаем обработанные данные с помощью `pickle`.

In [2]:
DATA_DIR = Path("../data/processed")

with open(DATA_DIR / "train.pkl", "rb") as fin:
    train_data = pickle.load(fin)
with open(DATA_DIR / "test.pkl", "rb") as fin:
    test_data = pickle.load(fin)

feature_names = set(train_data.keys())
assert all(feature in feature_names for feature in test_data.keys())
{name: train_data[name].shape for name in feature_names}

{'clicks': (12056598, 1),
 'zone_id': (12056598, 773),
 'banner_id': (12056598, 1054),
 'hour': (12056598, 1),
 'os_id': (12056598, 8),
 'coeff_sum0': (12056598, 1),
 'banner_id1': (12056598, 1054),
 'coeff_sum1': (12056598, 1),
 'g1': (12056598, 1),
 'weekday': (12056598, 1),
 'country_id': (12056598, 17),
 'g0': (12056598, 1)}

# Линейная модель

In [3]:
def construct_arrays(data: dict, use_banner_id1: bool = False
                     ) -> tuple[sparse.csr_array, sparse.csr_array]:
    features = ["weekday", "hour", "zone_id", "country_id", "os_id"]
    features.append("banner_id1" if use_banner_id1 else "banner_id")
    x = sparse.hstack([data[feature] for feature in features], format="csr")
    y = data["clicks"].ravel()
    return x, y

In [4]:
x_train, y_train = construct_arrays(train_data, False)
model = LogisticRegression(C=5.0, solver="liblinear")
model.fit(x_train, y_train)
train_loss = log_loss(y_train, model.predict_proba(x_train)[:, 1])
print(f"train loss: {train_loss}")

x_test, y_test = construct_arrays(test_data, False)
test_loss = log_loss(y_test, model.predict_proba(x_test)[:, 1])
print(f"test loss: {test_loss}")


train loss: 0.10232203793425765
test loss: 0.1338872469277786


Значения log loss получились примерно такими же, как и первом задании.